# EMO1 - areal time series
***

**_Autor:_** Chus Casado Rodríguez<br>
**_Fecha:_** 02-04-2024<br>

**Introduction:**<br>
This notebook computes the catchment averages of the meterological timeseries in EMO1. It can be deleted as it is much more efficient to carry out this task by using the new tool in `lisflood-utilities` called `catchstats`.

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
# import rioxarray
#from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import mapping
from typing import Union, List, Dict, Optional, Tuple
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf

from funciones import mask_statistics, plot_attributes

## Configuration

In [2]:
# path ResOpsES data set
path_datasets = Path('Z:/nahaUsers/casadje/datasets')
path_ResOpsES = path_datasets / 'reservoirs' / 'ResOpsES'

In [14]:
# path to EFASv5
path_EFAS = Path('E:/casadje/Documents/EFASv5/iberia')

In [3]:
# # coordinate reference system
# crs = 4326 # WGS84

## Reservoirs

In [4]:
# load reservoir points
reservoirs = gpd.read_file(path_ResOpsES / 'GIS' / 'reservoirs_ResOpsES_new.shp')
reservoirs.set_index('SNCZI', inplace=True)
reservoirs.index = reservoirs.index.astype(int)

reservoirs.shape

(305, 16)

## Catchments

In [7]:
# load catchment maps derived using `cutmaps`

path = path_ResOpsES / 'ancillary' / 'cutmaps'
mask_map = 'upArea_01min.nc'

masks = {}
directories = [dir for dir in path.iterdir() if dir.is_dir()]
for directory in tqdm(directories):
    try:
        # reservoir ID
        ID = int(directory.stem)
        
        # load upstream area map
        mask = xr.open_dataset(directory / mask_map)['Band1']
        mask.name = ID
        
        # create and save a mask out of the previous map
        mask = xr.where(mask.notnull(), 1, mask)
        masks[ID] = mask
    except:
        print(directory)
        continue

  0%|          | 0/307 [00:00<?, ?it/s]

Z:\nahaUsers\casadje\datasets\reservoirs\ResOpsES\ancillary\cutmaps\input
Z:\nahaUsers\casadje\datasets\reservoirs\ResOpsES\ancillary\cutmaps\masks


## Static maps

In [15]:
path_maps = path_EFAS / 'maps'

# cargar mapa de áreas de cada celda, que será utilizado en ciertos cálculos
pixarea = xr.open_mfdataset(f'{path_maps}/pixarea*.nc')['Band1'].compute()

# cargar mapa de áreas de cada celda, que será utilizado en ciertos cálculos
upArea = xr.open_mfdataset(f'{path_maps}/upArea*.nc')['Band1'].compute()

## Meteo

I've downloaded the daily data from EMO-1. However, EFASv5 uses the 6-hourly data.

In [20]:
path_meteo = path_EFAS / 'meteo'

# load lazily the meteorological data
variables = [x.stem for x in path_meteo.iterdir() if x.is_dir()]
meteo = {var: xr.open_mfdataset(f'{path_meteo}/{var}/EMO*{var}*.nc', chunks='auto', parallel=True)[var] for var in variables}
meteo = xr.Dataset(meteo)
meteo.close()

# recortar al periodo de estudio
#meteo = meteo.sel(time=slice(start, end))
# definir dimensiones con las coordenadas
#meteo = meteo.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
# definir sistema de coordenadas
#meteo = meteo.rio.write_crs(crs)

# convertir en volumen
# if var in ['pr', 'e0']:
#     da = da * area * 1e-3

# calcular medias areales
meteo_areal = mask_statistics(meteo, masks, func='mean', weight=pixarea)
meteo_areal = meteo_areal.rename_vars({'pr_mean': 'pr_emo1', 'ta_mean': 'ta_emo1', 'e0_mean': 'e0_emo1'})
del meteo

# export areal time series
path_out = path_ResOpsES / 'ancillary' / 'catchstats' / 'EMO1'
path_out.mkdir(parents=True, exist_ok=True)
for ID in tqdm(meteo_areal.id.data, desc='catchments'):
    meteo_areal.sel(id=ID).to_netcdf(path_out / f'{ID:04}.nc')